In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time

In [4]:
# URL
url = "https://u.gg/lol/profile/euw1/thehighground/champion-stats?queueType=normal_draft_5x5"

options = Options()
options.add_argument("--headless")
options.binary_location = "C:\\Users\\nicol\\Downloads\\chrome-win64\\chrome-win64\\chrome.exe"

service = Service("C:\\Users\\nicol\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)

driver.get(url)

wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.rt-tr-group")))

# Initialize list to store each row of data
data = []

# Get page source and create BeautifulSoup object
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Close the driver
driver.quit()

# Get all the rows using BeautifulSoup
rows = soup.select("div.rt-tr-group")

for i, row in enumerate(rows, 1):  # start from 1 because CSS nth-child starts from 1
    try:
        selectors = {
            'Champion': "div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > span:nth-child(2)",
            'WinsLoses': "div:nth-child(1) > div:nth-child(3) > div:nth-child(1) > span:nth-child(3)",
            'Winrate': "div:nth-child(1) > div:nth-child(3) > div:nth-child(1) > strong:nth-child(1)",
            'KDA': "div:nth-child(1) > div:nth-child(4) > div:nth-child(1) > div:nth-child(1) > strong:nth-child(1)",
            'KillsDeathsAssists': "div:nth-child(1) > div:nth-child(4) > div:nth-child(1) > span:nth-child(2)",
            'LP': "div:nth-child(1) > div:nth-child(5) > span:nth-child(1)",
            'MaxKills': "div:nth-child(1) > div:nth-child(6) > span:nth-child(1)",
            'MaxDeaths': "div:nth-child(1) > div:nth-child(7)",
            'CS': "div:nth-child(1) > div:nth-child(8) > span:nth-child(1)",
            'Damage': "div:nth-child(1) > div:nth-child(9) > span:nth-child(1)",
            'Gold': "div:nth-child(1) > div:nth-child(10) > span:nth-child(1)"
        }

        row_data = []
        for key, selector in selectors.items():
            element = row.select_one(selector)
            text = element.get_text(strip=True) if element else 'N/A'
            row_data.append(text)

        data.append(row_data)

    except Exception as e:
        print(f"Error in row {i}: {e}")

# Convert the data to a DataFrame
columns = ['Champion', 'WinsLoses', 'Winrate', 'KDA', 'KillsDeathsAssists', 'LP', 'MaxKills', 'MaxDeaths', 'CS', 'Damage', 'Gold']
df_individual = pd.DataFrame(data, columns=columns)

In [5]:
df_individual

,Champion,WinsLoses,Winrate,KDA,KillsDeathsAssists,LP,MaxKills,MaxDeaths,CS,Damage,Gold
0,Kha'Zix,2W 2L,50%,1.85,6.3/6.8/6.3,0LP,14,9,222.8,"20,886","13,996"
1,Ezreal,2W 1L,67%,2.25,3.3/4.0/5.7,0LP,6,9,213.3,"20,063","11,025"
2,Irelia,2W 1L,67%,1.53,3.7/5.7/5.0,0LP,9,11,259,"13,601","12,336"
3,Ornn,3W 0L,100%,3.67,5.0/4.0/9.7,0LP,8,7,182,"17,089","12,332"
4,Senna,1W 2L,33%,11.33,3.3/1.0/8.0,0LP,7,2,17.7,"11,458","7,212"
...,...,...,...,...,...,...,...,...,...,...,...
92,Xayah,1W 0L,100%,7.00,15.0/3.0/6.0,0LP,15,3,204,"20,553","14,024"
93,Yasuo,0W 1L,0%,1.00,2.0/9.0/7.0,0LP,2,9,264,"13,187","12,243"
94,Yone,0W 1L,0%,0.80,2.0/5.0/2.0,0LP,2,5,105,"7,392","6,530"
95,Yuumi,0W 1L,0%,1.20,0.0/5.0/6.0,0LP,0,5,9,"4,944","6,106"
